In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
import h5py
data_path = 'data/pca32v2/10M/dataset.h5'
%time f = h5py.File(data_path, 'r')

CPU times: user 1.21 ms, sys: 0 ns, total: 1.21 ms
Wall time: 3.75 ms


In [3]:
import h5py
data_path = 'data/clip768v2/10M/dataset.h5'
%time f = h5py.File(data_path, 'r')

CPU times: user 0 ns, sys: 1.01 s, total: 1.01 s
Wall time: 1.05 s


In [3]:
len(f['emb'])

10120191

## Goal:
1) load data for training (1000*1000 objects)
2) train, predict on train, dispose of the training data (keep ids)
3) load data for predict (rest 9M objects)
4) predict, dispose of the data (keep ids)
5) on search, load the data in the visited bucket again (10k objects on avg), compute distances

In [6]:
%time loaded_data = f['pca32'][:, :]

CPU times: user 19.1 ms, sys: 1.34 s, total: 1.36 s
Wall time: 12 s


In [7]:
data_path_32 = 'data/pca32v2/10M/dataset.h5'
%time f = h5py.File(data_path_32, 'r')
%time loaded_data_pca = f['pca32'][:, :]

CPU times: user 686 µs, sys: 0 ns, total: 686 µs
Wall time: 1.12 ms
CPU times: user 0 ns, sys: 618 ms, total: 618 ms
Wall time: 639 ms


In [8]:
from li.BulkLMI import BulkLMI
import pandas as pd

In [9]:
data = pd.DataFrame(loaded_data)
data.index += 1
data_s = data.sample(5_000_000, random_state=2023)

In [10]:
%%time
blmi = BulkLMI()
blmi.insert(data_s)

CPU times: user 985 ms, sys: 386 ms, total: 1.37 s
Wall time: 1.4 s


In [11]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s][%(levelname)-5.5s][%(name)-.20s] %(message)s'
)
LOG = logging.getLogger(__name__)


In [12]:
#clf = LogisticRegression(random_state=0).fit(X, y)

In [13]:
info_df = pd.DataFrame([], columns=['op', 'time-taken', 'size', '#-objects'])
%time info_df = blmi.deepen(blmi.nodes[(0, )], 100, info_df)

[2023-07-01 12:21:27,678][INFO ][li.BulkLMI.BulkLMI] ==== Deepen with (0,)
[2023-07-01 12:21:32,973][INFO ][numexpr.utils] Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2023-07-01 12:21:32,977][INFO ][numexpr.utils] Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


CPU times: user 52.6 s, sys: 4.71 s, total: 57.3 s
Wall time: 59.1 s


In [14]:
%%time
for i, leaf in enumerate(blmi.get_leaf_nodes_pos()):
    info_df = blmi.deepen(
        blmi.nodes[leaf],
        100,
        info_df
    )

[2023-07-01 12:22:27,203][INFO ][li.BulkLMI.BulkLMI] ==== Deepen with (0, 0)
[2023-07-01 12:22:27,913][INFO ][li.BulkLMI.BulkLMI] ==== Deepen with (0, 1)
[2023-07-01 12:22:28,472][INFO ][li.BulkLMI.BulkLMI] ==== Deepen with (0, 2)
[2023-07-01 12:22:29,398][INFO ][li.BulkLMI.BulkLMI] ==== Deepen with (0, 3)
[2023-07-01 12:22:30,167][INFO ][li.BulkLMI.BulkLMI] ==== Deepen with (0, 4)
[2023-07-01 12:22:30,729][INFO ][li.BulkLMI.BulkLMI] ==== Deepen with (0, 5)
[2023-07-01 12:22:31,663][INFO ][li.BulkLMI.BulkLMI] ==== Deepen with (0, 6)
[2023-07-01 12:22:32,361][INFO ][li.BulkLMI.BulkLMI] ==== Deepen with (0, 7)
[2023-07-01 12:22:32,997][INFO ][li.BulkLMI.BulkLMI] ==== Deepen with (0, 8)
[2023-07-01 12:22:33,509][INFO ][li.BulkLMI.BulkLMI] ==== Deepen with (0, 9)
[2023-07-01 12:22:34,231][INFO ][li.BulkLMI.BulkLMI] ==== Deepen with (0, 10)
[2023-07-01 12:22:34,777][INFO ][li.BulkLMI.BulkLMI] ==== Deepen with (0, 11)
[2023-07-01 12:22:35,689][INFO ][li.BulkLMI.BulkLMI] ==== Deepen with (0, 

CPU times: user 1min 37s, sys: 1.06 s, total: 1min 38s
Wall time: 1min 40s


In [15]:
blmi.dump_structure()

,type,children
key,,
"(0,)",InnerNode,100
"(0, 0)",InnerNode,100
"(0, 0, 0)",LeafNode,344
"(0, 0, 1)",LeafNode,361
"(0, 0, 2)",LeafNode,1336
...,...,...
"(0, 99, 95)",LeafNode,854
"(0, 99, 96)",LeafNode,321
"(0, 99, 97)",LeafNode,381


In [16]:
import numpy as np

In [17]:
data_to_insert = data.loc[np.setdiff1d(data.index, data_s.index)]
#blmi.insert(data_to_insert)
data_to_insert.shape

(5120191, 32)

In [18]:
%time blmi.insert(data_to_insert)

CPU times: user 15.2 s, sys: 4.36 s, total: 19.6 s
Wall time: 20.1 s


In [19]:
blmi.dump_structure()

,type,children
key,,
"(0,)",InnerNode,100
"(0, 0)",InnerNode,100
"(0, 0, 0)",LeafNode,711
"(0, 0, 1)",LeafNode,723
"(0, 0, 2)",LeafNode,2605
...,...,...
"(0, 99, 95)",LeafNode,1765
"(0, 99, 96)",LeafNode,701
"(0, 99, 97)",LeafNode,811


In [20]:
base_path = 'data/clip768v2/10M/'
base_path = 'data/pca32v2/10M/'

In [21]:
%%time
queries_path = f'{base_path}/query.h5'
f2 = h5py.File(queries_path, 'r')
#loaded_queries = f2['emb'][:, :]
loaded_queries = f2['pca32'][:, :]

CPU times: user 0 ns, sys: 3.22 ms, total: 3.22 ms
Wall time: 20.1 ms


In [22]:
%%time
gt_path = f'data/groundtruth-10M.h5'
f3 = h5py.File(gt_path, 'r')
loaded_gt = f3['knns'][:, :]

The history saving thread hit an unexpected error (OperationalError('disk I/O error',)).History will not be written to the database.
CPU times: user 0 ns, sys: 6.7 ms, total: 6.7 ms
Wall time: 52.8 ms


In [23]:
from tqdm import tqdm
from li.utils import pairwise_cosine


In [24]:
def increase_max_recursion_limit():
    """ Increases the maximum recursion limit.
    Source: https://stackoverflow.com/a/16248113
    """
    import sys
    import resource
    resource.setrlimit(resource.RLIMIT_STACK, (2**29, -1))
    sys.setrecursionlimit(10**6)
increase_max_recursion_limit()

In [25]:
%%time
res_all = []
for i, query in tqdm(enumerate(loaded_queries)):
    pred_leaf_nodes = blmi.search(query, stop_condition=1_000_000)[0]
    object_ids = []
    for pred_leaf_node in pred_leaf_nodes:
        leaf_node = blmi.nodes.get(pred_leaf_node)
        if leaf_node is not None:
            object_ids.extend(leaf_node.object_ids)

    #bucket_ids = [[i, mapping[bucket]] for bucket in blmi.search(query, stop_condition=500)[0]]
    dists = pairwise_cosine([query], data.loc[object_ids])
    break
    res_all.extend(bucket_ids)

0it [00:00, ?it/s]

CPU times: user 592 ms, sys: 67.6 ms, total: 660 ms
Wall time: 679 ms


In [26]:
len(object_ids)

1001425

In [27]:
import h5py
data_path = 'data/clip768v2/10M/dataset.h5'
%time f = h5py.File(data_path, 'r')
%time loaded_clip_data = f['emb'][:, :]

CPU times: user 638 µs, sys: 57 µs, total: 695 µs
Wall time: 3.95 ms
CPU times: user 20.8 ms, sys: 16 s, total: 16.1 s
Wall time: 2min 24s


In [28]:
loaded_clip_data.shape

(10120191, 768)

In [29]:
seq_data = pd.DataFrame(loaded_clip_data)
seq_data.index += 1

In [30]:
%%time
base_path = 'data/clip768v2/10M/'
queries_path = f'{base_path}/query.h5'
f2 = h5py.File(queries_path, 'r')
#loaded_queries = f2['emb'][:, :]
loaded_queries_seq = f2['emb'][:, :]

CPU times: user 0 ns, sys: 33.5 ms, total: 33.5 ms
Wall time: 291 ms


In [31]:
%time dists = pairwise_cosine([loaded_queries_seq[0]], seq_data.loc[object_ids])

CPU times: user 6.12 s, sys: 6.01 s, total: 12.1 s
Wall time: 12.6 s


In [32]:
k = 10

In [36]:
data.loc[object_ids].index[np.argsort(dists)[:, :k][0]].tolist()

[1831320,
 7535354,
 7059014,
 7563483,
 3003515,
 5278218,
 8190596,
 9060635,
 210839,
 5805329]

In [34]:
loaded_gt[0][:10]

array([1831320, 7535354, 7059014, 7563483, 3003515, 5278218, 8190596,
       9060635, 5509896,  210839], dtype=int32)

In [35]:
np.intersect1d(
    loaded_gt[0][:10],
    np.array(data.loc[object_ids].index[np.argsort(dists)[:, :k][0]])[:10]
).shape

(9,)

In [3]:
%time loaded_data = f['emb'][:10_000, :]

CPU times: user 3.23 ms, sys: 14.3 ms, total: 17.5 ms
Wall time: 372 ms


In [4]:
%%time
import numpy as np
rng = np.random.default_rng(2023)
random_idxs = rng.choice(range(10_000_000), size=(10_000), replace=False)
random_idxs = np.sort(random_idxs)

CPU times: user 1.22 s, sys: 284 ms, total: 1.51 s
Wall time: 1.59 s


In [7]:
%time loaded_data = f['emb'][random_idxs, :]

CPU times: user 2.17 s, sys: 784 ms, total: 2.95 s
Wall time: 1min 19s


In [3]:
%time _ = f['emb'][:100_000, :]

CPU times: user 13.7 ms, sys: 110 ms, total: 124 ms
Wall time: 2.05 s


In [4]:
%time _ = f['emb'][:1_000_000, :]

CPU times: user 12.7 ms, sys: 1.03 s, total: 1.05 s
Wall time: 20.7 s


In [5]:
%time _ = f['emb'][:9_000_000, :]

CPU times: user 11.6 ms, sys: 16.1 s, total: 16.1 s
Wall time: 3min 10s


In [8]:
%time pred_data = f['emb'][:, :]

CPU times: user 211 ms, sys: 14.5 s, total: 14.7 s
Wall time: 4min 4s


In [ ]:
pred_data[]

In [ ]:
%time test_mem = f['emb'][random_idxs, :]

In [20]:
%%time
rng = np.random.default_rng(2023)
print(rng)
random_pivots = rng.choice(range(10_000_000), size=(1_000), replace=False)
pivot_indexes = np.sort(random_pivots)
pivot_data = f['emb'][pivot_indexes, :]

Generator(PCG64)
CPU times: user 1.22 s, sys: 388 ms, total: 1.61 s
Wall time: 8.85 s


In [4]:
loaded_data.shape

(10000, 768)

In [73]:
from tqdm import tqdm
import gc
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import time

def pairwise_cosine(x, y):
    r = cosine_similarity(x, y)
    print(r)
    return 1-cosine_similarity(x, y)

In [28]:
pivot_data[0].unsqueeze()

AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'

In [36]:
loaded_data.shape

(1000, 768)

In [52]:
dists_all.shape

(2, 1000)

In [55]:
pivot_data.shape

(1000, 768)

In [59]:
training_data = np.empty((0,768))
training_data

array([], shape=(0, 768), dtype=float64)

In [ ]:
%%time
dists_all = np.empty((0,1000))
training_data = np.empty((0,768))
prev = 0

for batch, pivot in tqdm(zip(range(10_000, 10_000_000, 10_000), pivot_data)):
    loaded_data = f['emb'][prev:batch, :]
    dists = np.argsort(cosine_similarity([pivot], loaded_data)[0])
    training_data_indexes = dists[-1000:]
    # take the 1000 most similar (argsort sorts from lowest to highest)
    dists_all = np.vstack((dists_all, prev+training_data_indexes))
    training_data = np.vstack((training_data, loaded_data[training_data_indexes]))
    del loaded_data
    gc.collect()
    prev = batch

In [75]:
training_data.shape

(999000, 768)

In [ ]:
[]

In [95]:
%%time
labels = np.array([np.array([i for j in range(1000)]) for i in range(999)])
labels = labels.reshape(labels.shape[0]*labels.shape[1])

CPU times: user 106 ms, sys: 2.27 ms, total: 108 ms
Wall time: 121 ms


In [69]:
dists_all.shape

(10, 1000)

In [87]:
from typing import Tuple
def get_device() -> torch.device:
    """ Gets the `device` to be used by torch.
    This arugment is needed to operate with the PyTorch model instance.

    Returns
    ------
    torch.device
        Device
    """
    use_cuda = torch.cuda.is_available()
    device = torch.device('cuda:0' if use_cuda else 'cpu')
    torch.backends.cudnn.benchmark = True
    return device


def data_X_to_torch(data) -> torch.FloatTensor:
    """ Creates torch training data."""
    data_X = torch.from_numpy(np.array(data).astype(np.float32))
    return data_X


def data_to_torch(data, labels) -> Tuple[torch.FloatTensor, torch.LongTensor]:
    """ Creates torch training data and labels."""
    data_X = data_X_to_torch(data)
    data_y = torch.as_tensor(torch.from_numpy(labels), dtype=torch.long)
    return data_X, data_y

In [113]:
import torch
from torch import nn

class Model(nn.Module):
    def __init__(self, input_dim=768, output_dim=1000):
        super().__init__()
        self.layers = torch.nn.Sequential(
          torch.nn.Linear(input_dim, 128),
          torch.nn.ReLU(),
          torch.nn.Linear(128, output_dim)
        )

    def forward(self, x: torch.FloatTensor) -> torch.FloatTensor:
        outputs = self.layers(x)
        return outputs

model = Model()
lr = 0.01
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
device = get_device()

In [98]:
%%time
data_X, data_y = data_to_torch(training_data, labels)
data_X.shape, data_y.shape

CPU times: user 756 ms, sys: 741 ms, total: 1.5 s
Wall time: 1.6 s


(torch.Size([999000, 768]), torch.Size([999000]))

In [135]:
def train(
    data_X: torch.FloatTensor,
    data_y: torch.LongTensor,
    model,
    optimizer,
    device,
    epochs=500,
):
    step = data_X.shape[0] // epochs // 10
    losses = []
    for ep in range(epochs):
        if ep % step == 0 and ep != 0:
            print(f'{time.time()} | Epoch {ep} | Loss {curr_loss.item()}')
        pred_y = model(data_X.to(device))
        curr_loss = loss(pred_y, data_y.to(device))
        losses.append(curr_loss.item())

        model.zero_grad()
        curr_loss.backward()

        optimizer.step()
    return losses, model


In [111]:
%%time
rng_offset = 1
rng = np.random.default_rng(2023+rng_offset)
rng_offset += 1
print(rng)
random_idxs = rng.choice(range(999000), size=(10_000), replace=False)

Generator(PCG64)
CPU times: user 99 ms, sys: 39.6 ms, total: 139 ms
Wall time: 139 ms


In [112]:
data_X_s = data_X[random_idxs]
data_y_s = data_y[random_idxs]
data_X_s.shape, data_y_s.shape

(torch.Size([10000, 768]), torch.Size([10000]))

In [136]:
model = Model(output_dim=np.unique(data_y_s).shape[0])
lr = 0.1
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
device = get_device()

In [137]:
%time losses, model = train(data_X_s, data_y_s, model, optimizer, device, epochs=100)

Epoch 10
Epoch 20
Epoch 30
Epoch 40
Epoch 50
Epoch 60
Epoch 70
Epoch 80
Epoch 90
CPU times: user 55.6 s, sys: 2.76 s, total: 58.3 s
Wall time: 59.8 s


In [141]:
def predict( model, device, data_X: torch.FloatTensor):
    """ Collects predictions for multiple data points (used in structure building)."""
    model = model.to(device)
    model.eval()

    all_outputs = torch.tensor([], device=device)
    with torch.no_grad():
        outputs = model(data_X.to(device))
        all_outputs = torch.cat((all_outputs, outputs), 0)

    _, y_pred = torch.max(all_outputs, 1)
    return y_pred.cpu().numpy()

In [142]:
predictions = predict(model, device, data_X_s)

In [146]:
data_y_s

tensor([515, 589, 280,  ..., 141, 934,  88])

In [145]:
predictions

array([515, 589, 280, ..., 141, 934,  88])

### we can let go of the training data

In [147]:
del training_data
gc.collect()

67

In [ ]:
# load everyting and then filter out whatever
%time pred_data = f['emb'][:, :]

In [157]:
%%time
predict_indexes = np.setdiff1d(np.arange(0, 10_000_000), dists_all.flatten())
predict_indexes.shape

CPU times: user 306 ms, sys: 367 ms, total: 672 ms
Wall time: 813 ms


(9001000,)

In [158]:
%%time
predict_indexes = np.sort(predict_indexes)
predict_indexes.shape

CPU times: user 102 ms, sys: 67.5 ms, total: 170 ms
Wall time: 172 ms


(9001000,)

In [159]:
predict_indexes[:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
%%time
loaded_pred_data = f['emb'][predict_indexes, :]
loaded_pred_data.shape

In [ ]:
loaded_pred_data[:10]

In [155]:
dists_all.reshape((dists_all[0]*dists_all[1]))#.shape

ValueError: maximum supported dimension for an ndarray is 32, found 1000

In [ ]:
    def predict(self, data_X: torch.FloatTensor):
        """ Collects predictions for multiple data points (used in structure building)."""
        self.model = self.model.to(self.device)
        self.model.eval()

        all_outputs = torch.tensor([], device=self.device)
        with torch.no_grad():
            outputs = self.model(data_X.to(self.device))
            all_outputs = torch.cat((all_outputs, outputs), 0)

        _, y_pred = torch.max(all_outputs, 1)
        return np.array([self.model.output_neurons[label] for label in y_pred.cpu().numpy()])

    def predict_single(self, data_X: torch.FloatTensor):
        """ Collects predictions for a single data point (used in query predictions)."""
        self.model = self.model.to(self.device)
        self.model.eval()

        with torch.no_grad():
            outputs = self.model(data_X.to(self.device))

        prob = nnf.softmax(outputs, dim=0)
        top_prob, top_class = prob.topk(self.model.n_output_neurons, dim=0)
        top_prob = top_prob.cpu().numpy()
        return top_prob, np.array([self.model.output_neurons[label] for label in top_class.cpu().numpy()])
